In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession
#Initilize Sparksession
spark = SparkSession.builder.appName("WordCount").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 03:45:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
rdd=spark.sparkContext.parallelize([1,2,3,4])

In [5]:
rdd.collect()

[1, 2, 3, 4]

In [6]:
resultrdd = rdd.map(lambda x:x*2)

In [7]:
resultrdd.collect()

[2, 4, 6, 8]

In [8]:
rdd1 = spark.sparkContext.parallelize([1,2,3,4,5])

In [9]:
resultrdd1 = rdd1.flatMap(lambda x:(x,x*2))
resultrdd1.collect()

[1, 2, 2, 4, 3, 6, 4, 8, 5, 10]

In [10]:
rdd3 = spark.sparkContext.parallelize([1,2,3,4,5,6,7])

In [11]:
resultrdd3=rdd3.filter(lambda x:x % 2==0)
resultrdd3.collect()

[2, 4, 6]

In [12]:
rdd4 = spark.sparkContext.parallelize([(1,2),(3,4),(1,6),(2,3)])
res4 = rdd4.reduceByKey(lambda x,y : x+y)
res4.collect()

[(2, 3), (1, 8), (3, 4)]

In [13]:
rdd5 = spark.sparkContext.parallelize([(1,2),(3,4),(1,6),(2,3)])
res5 = rdd5.groupByKey()
res5.collect()

[(2, <pyspark.resultiterable.ResultIterable at 0x7f57bc1cbb50>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f57bc1ca610>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7f57bc1cbcd0>)]

In [14]:
for key, values in res5.collect():
    print(f"Key: {key}, Values:{list(values)}")

Key: 2, Values:[3]
Key: 1, Values:[2, 6]
Key: 3, Values:[4]


In [15]:
word_list = ["this", "is", "a", "sample", "text", "document", "for", "word", "count", "example", "word", "count"]
rdd = spark.sparkContext.parallelize(word_list)
words_count = rdd.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
results = words_count.collect()
for word, count in results:
    print(f"{word}: {count}")

this: 1
sample: 1
text: 1
for: 1
word: 2
is: 1
a: 1
document: 1
count: 2
example: 1


In [16]:
purchaserdd = spark.sparkContext.textFile("/home/labuser/Documents/Pandas_datasets/purchases.csv")
purchaserdd.collect()

[',apples,oranges', 'June,3,0', 'Robert,2,3', 'Lily,0,7', 'David,1,2']

In [17]:
purchasedf=spark.read.csv("/home/labuser/Documents/Pandas_datasets/purchases.csv")

In [18]:
purchasedf.show()

+------+------+-------+
|   _c0|   _c1|    _c2|
+------+------+-------+
|  null|apples|oranges|
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



In [19]:
purchasedf.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [20]:
purchasedf_01=spark.read.option("inferschema", True).option("header", True).csv("/home/labuser/Documents/Pandas_datasets/purchases.csv")
purchasedf_01.show()

+------+------+-------+
|   _c0|apples|oranges|
+------+------+-------+
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



23/09/22 03:46:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , apples, oranges
 Schema: _c0, apples, oranges
Expected: _c0 but found: 
CSV file: file:///home/labuser/Documents/Pandas_datasets/purchases.csv


In [21]:
purchasedf_01.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- apples: integer (nullable = true)
 |-- oranges: integer (nullable = true)



In [24]:
moviedf=spark.read.schema(movieSchema).option("header", True).csv("/home/labuser/Documents/Pandas_datasets/IMDB-Movie-Data.csv")
moviedf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [23]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
movieSchema=StructType([StructField("Rank", IntegerType(),True),
                        StructField("Title", StringType(),True),
                        StructField("Genre", StringType(),True),
                        StructField("Description", StringType(),True),
                        StructField("Director", StringType(),True),
                        StructField("Actors", StringType(),True),
                        StructField("Year", StringType(),True),
                        StructField("Runtime (Minutes)", StringType(),True),
                        StructField("Rating", StringType(),True),
                        StructField("Votes", StringType(),True),
                        StructField("Revenue (Millions)", DoubleType(),True),
                        StructField("Metascore", DoubleType(),True)])

In [25]:
moviedf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|              124|     7|485820|            126.46|     65.0|
|   3|               Split|   

In [26]:
from pyspark.sql.functions import *
moviedf = moviedf.withColumn("rev_col",col("Revenue (Millions)")*100)

In [27]:
moviedf.describe()

DataFrame[summary: string, Rank: string, Title: string, Genre: string, Description: string, Director: string, Actors: string, Year: string, Runtime (Minutes): string, Rating: string, Votes: string, Revenue (Millions): string, Metascore: string, rev_col: string]

In [28]:
moviedf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|          rev_col|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|          33313.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|              124|  

In [29]:
from pyspark.sql.functions import *
moviedf = moviedf.withColumn("rev_col",col("Revenue (Millions)")*100).withColumn("Batch", lit("Batch3"))
moviedf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|          rev_col| Batch|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|          33313.0|Batch3|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|             

In [30]:
moviedf.rdd.getNumPartitions()

1

In [31]:
newdf = moviedf.repartition(10)

In [33]:
newdf.rdd.getNumPartitions()

10

In [35]:
newdf.write.csv('/home/labuser/Documents/test/Partitions')

In [36]:
spark_ui_url = f'{spark._jsc.sc().uiWebUrl().get()}/'
print("Spark UI URL:", spark_ui_url)

Spark UI URL: http://ip-172-31-1-230.ap-south-1.compute.internal:4040/


In [40]:
moviedf.createOrReplaceTempView("movie")

In [41]:
result = spark.sql("select * from movie")
type(result)

pyspark.sql.dataframe.DataFrame

In [42]:
result.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|          rev_col| Batch|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|          33313.0|Batch3|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|             

In [43]:
testdf=moviedf.sort(col("Title").desc())
testdf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+------------------+------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|           rev_col| Batch|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+------------------+------+
|  75|            Zootopia|Animation,Adventu...|In a city of anth...|        Byron Howard|Ginnifer Goodwin,...|                2016|              108|   8.1|296853|            341.26|     78.0|           34126.0|Batch3|
| 432|         Zoolander 2|              Comedy|Derek and Hansel ...|         Ben Stiller|Ben Stiller, Owen...|         

In [45]:
df=moviedf.select('Rank','Title','Genre')
df.show()

+----+--------------------+--------------------+
|Rank|               Title|               Genre|
+----+--------------------+--------------------+
|   1|Guardians of the ...|Action,Adventure,...|
|   2|          Prometheus|Adventure,Mystery...|
|   3|               Split|     Horror,Thriller|
|   4|                Sing|Animation,Comedy,...|
|   5|       Suicide Squad|Action,Adventure,...|
|   6|      The Great Wall|Action,Adventure,...|
|   7|          La La Land|  Comedy,Drama,Music|
|   8|            Mindhorn|              Comedy|
|   9|  The Lost City of Z|Action,Adventure,...|
|  10|          Passengers|Adventure,Drama,R...|
|  11|Fantastic Beasts ...|Adventure,Family,...|
|  12|      Hidden Figures|Biography,Drama,H...|
|  13|           Rogue One|Action,Adventure,...|
|  14|               Moana|Animation,Adventu...|
|  15|            Colossal| Action,Comedy,Drama|
|  16|The Secret Life o...|Animation,Adventu...|
|  17|       Hacksaw Ridge|Biography,Drama,H...|
|  18|        Jason 

In [46]:
from datetime import *
df=df.withColumn("last_updated_ts", lit(datetime.now()))
df.show()

+----+--------------------+--------------------+--------------------+
|Rank|               Title|               Genre|     last_updated_ts|
+----+--------------------+--------------------+--------------------+
|   1|Guardians of the ...|Action,Adventure,...|2023-09-22 05:32:...|
|   2|          Prometheus|Adventure,Mystery...|2023-09-22 05:32:...|
|   3|               Split|     Horror,Thriller|2023-09-22 05:32:...|
|   4|                Sing|Animation,Comedy,...|2023-09-22 05:32:...|
|   5|       Suicide Squad|Action,Adventure,...|2023-09-22 05:32:...|
|   6|      The Great Wall|Action,Adventure,...|2023-09-22 05:32:...|
|   7|          La La Land|  Comedy,Drama,Music|2023-09-22 05:32:...|
|   8|            Mindhorn|              Comedy|2023-09-22 05:32:...|
|   9|  The Lost City of Z|Action,Adventure,...|2023-09-22 05:32:...|
|  10|          Passengers|Adventure,Drama,R...|2023-09-22 05:32:...|
|  11|Fantastic Beasts ...|Adventure,Family,...|2023-09-22 05:32:...|
|  12|      Hidden F

In [47]:
df_test=df.selectExpr('CAST(last_updated_ts AS DATE) AS test', 'Rank', 'Title', 'Genre')
df_test.show()

+----------+----+--------------------+--------------------+
|      test|Rank|               Title|               Genre|
+----------+----+--------------------+--------------------+
|2023-09-22|   1|Guardians of the ...|Action,Adventure,...|
|2023-09-22|   2|          Prometheus|Adventure,Mystery...|
|2023-09-22|   3|               Split|     Horror,Thriller|
|2023-09-22|   4|                Sing|Animation,Comedy,...|
|2023-09-22|   5|       Suicide Squad|Action,Adventure,...|
|2023-09-22|   6|      The Great Wall|Action,Adventure,...|
|2023-09-22|   7|          La La Land|  Comedy,Drama,Music|
|2023-09-22|   8|            Mindhorn|              Comedy|
|2023-09-22|   9|  The Lost City of Z|Action,Adventure,...|
|2023-09-22|  10|          Passengers|Adventure,Drama,R...|
|2023-09-22|  11|Fantastic Beasts ...|Adventure,Family,...|
|2023-09-22|  12|      Hidden Figures|Biography,Drama,H...|
|2023-09-22|  13|           Rogue One|Action,Adventure,...|
|2023-09-22|  14|               Moana|An

In [49]:
df_test=df_test.withColumn("Movie_Name", lit(col("Title")))
df_test.show()

+----------+----+--------------------+--------------------+--------------------+
|      test|Rank|               Title|               Genre|          Movie_Name|
+----------+----+--------------------+--------------------+--------------------+
|2023-09-22|   1|Guardians of the ...|Action,Adventure,...|Guardians of the ...|
|2023-09-22|   2|          Prometheus|Adventure,Mystery...|          Prometheus|
|2023-09-22|   3|               Split|     Horror,Thriller|               Split|
|2023-09-22|   4|                Sing|Animation,Comedy,...|                Sing|
|2023-09-22|   5|       Suicide Squad|Action,Adventure,...|       Suicide Squad|
|2023-09-22|   6|      The Great Wall|Action,Adventure,...|      The Great Wall|
|2023-09-22|   7|          La La Land|  Comedy,Drama,Music|          La La Land|
|2023-09-22|   8|            Mindhorn|              Comedy|            Mindhorn|
|2023-09-22|   9|  The Lost City of Z|Action,Adventure,...|  The Lost City of Z|
|2023-09-22|  10|          P

In [50]:
moviedf.count()

1000

In [51]:
newdf_01 = moviedf.dropDuplicates(["Director"])

In [52]:
newdf_01.count()

646

In [60]:
test_df=moviedf.withColumn("Rating_Value", when((col("Rating")>=0.0) & (col("Rating")<5.0), "Average"). 
                           when((col("Rating")>=5.0) & (col("Rating")<8.0), "Good").
                           when((col("Rating")>=8.0) & (col("Rating")<=10.0), "Best").
                           otherwise("NA"))
test_df.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+------------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|          rev_col| Batch|Rating_Value|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|          33313.0|Batch3|        Best|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|   

In [61]:
print("hello")

hello


In [64]:
test_df = test_df.withColumn("Concatenated", concat(col("Title"), lit("_Shell")))

In [65]:
test_df.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+------------+--------------------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|          rev_col| Batch|Rating_Value|        Concatenated|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+------------+--------------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|          33313.0|Batch3|        Best|Guardians 

In [66]:
def concat_shell(column):
    return column + "_shell"

In [67]:
my_udf = udf(concat_shell, StringType())

In [70]:
from pyspark.sql.functions import *
test = test_df.withColumn("new_col", my_udf(col("Title")))

In [71]:
test.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+------------+--------------------+--------------------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|          rev_col| Batch|Rating_Value|        Concatenated|             new_col|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+------------+--------------------+--------------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            33

In [72]:
test_df = test_df.drop('new_col')

In [73]:
test_df.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+------------+--------------------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|          rev_col| Batch|Rating_Value|        Concatenated|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+------------+--------------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|          33313.0|Batch3|        Best|Guardians 

In [74]:
new_df = moviedf.cache()

In [75]:
new_df.show(2)

+----+--------------------+--------------------+--------------------+------------+--------------------+----+-----------------+------+------+------------------+---------+-------+------+
|Rank|               Title|               Genre|         Description|    Director|              Actors|Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|rev_col| Batch|
+----+--------------------+--------------------+--------------------+------------+--------------------+----+-----------------+------+------+------------------+---------+-------+------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|  James Gunn|Chris Pratt, Vin ...|2014|              121|   8.1|757074|            333.13|     76.0|33313.0|Batch3|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|Ridley Scott|Noomi Rapace, Log...|2012|              124|     7|485820|            126.46|     65.0|12646.0|Batch3|
+----+--------------------+--------------------+--------------------+------